In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
crunch = ["#10069F", "#11FF9B", "#7F7F7F", "#E10098","#E7E6E6", "#440099"] # Blue - Green - Gray - Pink - Light Grey - Purple
sns.set_palette(crunch)
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 10

In [ ]:
def normalize_per_row(df:pd.DataFrame)->pd.DataFrame:
  return df.apply(lambda row:row/row.sum() if (row.sum() != 0) else row, axis=1)

In [ ]:
df_sales = pd.read_gbq("SELECT * FROM `everest-testing-ground.raw_data.sales`", project_id="everest-testing-ground", use_bqstorage_api=True)
df_brands = pd.read_gbq("SELECT * FROM `everest-testing-ground.raw_data.brandmap`", project_id="everest-testing-ground", use_bqstorage_api=True)
df_to_distribute = pd.read_gbq("SELECT * FROM `everest-testing-ground.raw_data.to_distribute`", project_id="everest-testing-ground", use_bqstorage_api=True)
df_product_groups = pd.read_gbq("SELECT * FROM `everest-testing-ground.raw_data.productgroupmap`", project_id="everest-testing-ground", use_bqstorage_api=True)

df_sales=df_sales.merge(df_brands, on='product_id',how='left')

In [ ]:
# Compute sales totals per product per shop
df_product_sales = df_sales.groupby(['product_id', 'stock_location_id'])['sales_amount'].sum().unstack(fill_value=0)

# Compute sales totals per brand per shop
df_brand_sales = df_sales.groupby(['brand', 'stock_location_id'])['sales_amount'].sum().unstack(fill_value=0)

# Distributionmatrix based on products sold per shop
df_productlevel = df_to_distribute.drop('initial_total_stock', axis=1).merge(df_product_sales, on='product_id', how='left').fillna(0).set_index('product_id')

# Distributionmatrix based on brands sold per shop
df_brandlevel = df_to_distribute.drop('initial_total_stock', axis=1).merge(df_brands, on='product_id', how='left').merge(df_brand_sales, on='brand', how='left').fillna(0).drop('brand', axis=1).set_index('product_id')

# Distributionmatrix based on items sold per shop
df_shoplevel = pd.DataFrame(index=df_to_distribute.product_id,columns=df_brandlevel.columns) 
for shop in df_shoplevel.columns:
  df_shoplevel[shop] = df_sales.loc[df_sales.stock_location_id==shop]['sales_amount'].sum()

# Normalize data
df_productlevel = normalize_per_row(df_productlevel)
df_brandlevel = normalize_per_row(df_brandlevel)
df_shoplevel = normalize_per_row(df_shoplevel) 

display(df_productlevel)
display(df_brandlevel)
display(df_shoplevel)

In [ ]:
w_p = 5 # Weight of the productlevel
w_b = 6 # Weight of the brandlevel
w_s = 10 # Weight of the shoplevel

# Add all normalized and weighted distribution matrices
df_dist = w_p*df_productlevel + w_b*df_brandlevel + w_s*df_shoplevel 

# Normalize end result per row
df_dist = normalize_per_row(df_dist)

# Final distribution matrix
display(df_dist) 

In [ ]:
# Calculate final distribution based on distributionmatrix

df_result = df_dist.mul(df_to_distribute.set_index('product_id')['initial_total_stock'], axis=0).round()
df_result.columns = [f"shop_{col}" for col in df_result.columns]
display(df_result)

In [ ]:
test_product = df_result.sample()
plt.bar(test_product.columns.astype(str),test_product.iloc[0].values)
plt.title("Product: " + test_product.index[0]+", # to distribute: " + str(int(test_product.sum(axis=1))))

In [ ]:
df_result.reset_index().to_gbq(destination_table="dataset_output.product_distributions")